### Phys 629: Statistical Tools for Physics Research
***Anuradha Gupta***

# Homework 8
### Due: Friday, Oct 27 at 11:59 pm CT

## Problem 1

This week's problem is worth 40 points. This problem uses a dataset in `/coursework/homeworks/hw_data/`.

1) Read in `hw8_data.npy`, which is a (3 x 20) numpy array. Set `x, y, sigma_y = data`. 

We're going to do some polynomial fits to this data just like in the lecture. However, in all cases you should **keep the $y$-intercept fixed at $-0.23$**. 

2) Use the following code to compute the un-normalized posterior pdf (i.e., just the likelihood x prior) on a grid of the linear coefficient (i.e., the slope) of a linear model, with a uniform prior between 0.5 and 1.5. Plot this posterior pdf. Remember this is just a one-dimensional model because the $y$-intercept is fixed. I advise a grid size of 100.

In [8]:
#Functions to do a polynomial fit, and compute the likelihood
def polynomial_fit(theta, x):
    """Polynomial model of degree (len(theta) - 1)"""
    # For a polynomial with order 1, this gives theta_0 + theta_1*x
    # For a polynomial with order 2, this gives theta_0 + theta_1*x + theta_2*x^2, etc.
    return sum(t * x ** n for (n, t) in enumerate(theta))

# compute the data log-likelihood given a model
def logL(theta, data, model=polynomial_fit):
    """Gaussian log-likelihood of the model at theta"""
    x, y, sigma_y = data
    y_fit = model(theta, x)
    return sum(scipy.stats.norm.logpdf(*args) 
               for args in zip(y, y_fit, sigma_y))

3) Using your 1D gridded likelihood x prior, compute the Bayesian evidence of this linear model. This may be a big number!

4) Now compute the joint 2D posterior pdf (again just the likelihood x prior) of linear and quadratic coefficients of a quadratic model. Give the linear coefficient a uniform prior between 0.5 and 1.5. Give the quadratic coefficient a uniform prior between -1 and 0.25. Plot this 1D posterior. Remember this is a two-dimensional model because the $y$-intercept is fixed. I advise a grid size of 100 in each model parameter dimension.

5) Using your 2D gridded likelihood x prior, compute the Bayesian evidence of the quadratic model. 

In [1]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'
from IPython.display import display, Latex

import scipy.stats
from scipy.stats import norm
from scipy.stats import uniform
from scipy import optimize
from scipy import integrate
import pandas as pd 

In [7]:
x, y, sigma_y = np.load('../../homeworks/hw_data/hw8_data.npy')

In [ ]:
c = -0.23

def prior1D(y, loc=1, scale=1):
    return 1/scale

def post(y):
    return np.exp(logL(theta, data, model=polynomial_fit)) * () 